In [1]:
import pandas as pd
import nltk
import spacy
import gensim
import seaborn as sb
from gensim import corpora, models, similarities
from spacy.lang.en import English
from nltk.corpus import wordnet as wn

In [2]:
data=pd.read_csv("MecklenburgCounty_original_data.csv",)

In [3]:
data.head()

,Unnamed: 0,page_number,word
0,1,1,meckncgov
1,2,1,fiscal
2,3,1,year
3,4,1,adopted
4,5,1,budget


In [4]:
data.drop(data.columns[0], axis=1)

,page_number,word
0,1,meckncgov
1,1,fiscal
2,1,year
3,1,adopted
4,1,budget
5,1,mecklenburg
6,1,county
7,1,north
8,1,carolina
9,2,adopted


In [5]:
data.describe()

,Unnamed: 0,page_number
count,84235.0000,84235.000000
mean,42118.0000,260.653505
std,24316.6943,171.952653
min,1.0000,1.000000
25%,21059.5000,98.000000
50%,42118.0000,252.000000
75%,63176.5000,416.000000
max,84235.0000,537.000000


In [6]:
#cleaning 
spacy.load('en')
parser=English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.orth_.startswith('http'):
            lda_tokens.append('com')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [7]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def get_lemma(word):
    lemma=wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [9]:
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [10]:
#filtering stopwords
nltk.download('stopwords')
stop=set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\messi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
#preparing for lda
def prepare_lda(text):
    tokens=tokenize(text)
    tokens=[token for token in tokens if len(token)>4]
    tokens=[token for token in tokens if token not in stop]
    tokens=[get_lemma(token) for token in tokens]
    return tokens

In [12]:
#creating tokens
import random
text_data=[]
with open('GuilfordCounty_original_data.csv') as f:
    for line in f:
        tokens=prepare_lda(line)
        if random.random()>.99:
            print(tokens)
            text_data.append(tokens)

['181",2,"instruction']
['491",5,"are']
['500",5,"for']
['755",9,"support']
['910",11,"foster']
['934",11,"present']
['1370",12,"the']
['1380",12,"constraints']
['1483",12,"community']
['1505",12,"had']
['1532",12,"this']
['1566",12,"and']
['1593",12,"fund']
['1688",13,"the']
['1731",13,"at']
['1774",13,"amount']
['1828",13,"the']
['2038",14,"by']
['2244",15,"in']
['2352",15,"o']
['2416",15,"means']
['2420",15,"some']
['2634",16,"rates']
['2708",16,"as']
['2755",16,"annual']
['2763",16,"animal']
['2830",16,"less']
['2916",16,"restricted']
['2962",17,"fy']
['3151",17,"s']
['3189",18,"a']
['3314",18,"needs']
['3443",18,"fy']
['3445",18,"county']
['3479",18,"within']
['3480",18,"the']
['3594",18,"will']
['3681",18,"funds']
['3759",19,"these']
['3763",19,"the']
['3913",19,"transformation']
['4017",19,"services']
['4202",20,"are']
['4211",20,"county']
['4246",20,"disposable']
['4574",21,"one']
['4727",21,"of']
['4954",21,"in']
['4993",22,"have']
['5040",22,"allow']
['5197",22,"million']
['5

['38300",175,"survival']
['38309",175,"and']
['38590",176,"with']
['38875",178,"time']
['39200",179,"our']
['39211",179,"continue']
['39277",179,"communications']
['39304",179,"recruit']
['39353",180,"economic']
['39378",180,"and']
['39638",181,"standard']
['39748",181,"and']
['39779",181,"citizens']
['39817",182,"revenues']
['39862",182,"by']
['39866",182,"accurately']
['39972",182,"collaborated']
['40578",185,"departmental']
['40581",185,"procedures']
['40637",185,"upon']
['40661",185,"and']
['40681",185,"layer']
['40691",185,"system']
['40766",186,"expense']
['40940",187,"with']
['41272",188,"ensure']
['41328",188,"funds']
['41426",188,"of']
['41461",188,"when']
['41495",188,"the']
['41711",189,"performance']
['41754",189,"conceptual']
['41838",189,"nc']
['41901",189,"actual']
['41909",189,"hours']
['41919",189,"to']
['42091",190,"the']
['42406",192,"funds']
['42459",192,"these']
['42630",193,"expenses']
['42649",193,"in']
['42667",193,"program']
['42681",193,"monitored']
['42802",1

['72775",335,"exits']
['72827",336,"concrete']
['72970",337,"fy']
['73363",340,"body']
['73466",341,"budget']
['73534",341,"new']
['73564",341,"services']
['73873",343,"processes']
['73963",343,"fall']
['74027",344,"a']
['74086",344,"project']
['74114",344,"board']
['74197",344,"by']
['74230",344,"from']
['74298",344,"phases']
['74326",344,"budgets']
['74530",345,"of']
['74795",346,"for']
['74842",347,"iii']
['74862",347,"guilford']
['74966",347,"not']
['75203",348,"the']
['75578",350,"golf']
['75654",350,"voting']
['75676",351,"funds']
['75931",353,"housedbudgeted']
['75970",354,"i']
['76086",354,"work']
['76199",356,"appropriated']
['76359",357,"capital']
['76406",358,"nc']
['76565",359,"description']
['76569",359,"or']
['76816",359,"fy']
['76866",359,"needed']
['76875",359,"total']
['76978",360,"of']
['77022",360,"funded']
['77038",360,"received']
['77100",360,"to']
['77101",360,"a']
['77212",361,"zenke']
['77396",362,"the']
['77429",362,"in']
['77449",362,"parking']
['77528",362,"t

In [13]:
#creating dictionary
from gensim import corpora
dictionary=corpora.Dictionary(text_data)
corpus= [dictionary.doc2bow(text) for text in text_data]
import pickle 
pickle.dump(corpus, open('corpus.pkl', 'wb'))

In [14]:
#example
ldamodel=gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=6)
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} word: {}'.format(idx, topic))

Topic: 0 word: 0.006*"68207",307,"fire" + 0.006*"87477",449,"district" + 0.006*"41426",188,"of" + 0.006*"68808",312,"the" + 0.006*"65193",287,"be" + 0.006*"20533",96,"to" + 0.006*"88241",454,"the" + 0.006*"73534",341,"new" + 0.006*"36779",170,"resources" + 0.006*"27459",128,"through"
Topic: 1 word: 0.006*"88764",458,"reclassifications" + 0.006*"65156",287,"measures" + 0.006*"19269",90,"is" + 0.006*"2755",16,"annual" + 0.006*"43037",196,"or" + 0.006*"48967",220,"total" + 0.006*"28171",132,"has" + 0.006*"81582",389,"plans" + 0.006*"67378",303,"enough" + 0.006*"43436",199,"or"
Topic: 2 word: 0.006*"59920",264,"to" + 0.006*"8100",34,"manager" + 0.006*"53310",240,"needs" + 0.006*"56514",253,"fy" + 0.006*"491",5,"are" + 0.006*"16866",78,"an" + 0.006*"1774",13,"amount" + 0.006*"26864",126,"well" + 0.006*"45902",209,"demand" + 0.006*"39862",182,"by"
Topic: 3 word: 0.006*"11872",49,"deficiency" + 0.006*"66394",294,"use" + 0.006*"29298",136,"validation" + 0.006*"70940",324,"exterior" + 0.006*"57

In [15]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\messi\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
